<h2><center>Real Estate House Price Prediction Application</center></h2>


House Features

    INT_SQFT – The interior Sq. Ft of the property
    N_BEDROOM – The number of Bed rooms
    N_BATHROOM - The number of bathrooms
    N_ROOM – Total Number of Rooms
    QS_ROOMS – The quality score assigned for rooms based on buyer reviews
    QS_BATHROOM – The quality score assigned for bathroom based on buyer reviews
    QS_BEDROOM – The quality score assigned for bedroom based on buyer reviews
    QS_OVERALL – The Overall quality score assigned for the property
    SALE_COND – The Sale Condition
        Normal: Normal Sale
        Abnorml: Abnormal Sale - trade, foreclosure, short sale
        AdjLand: Adjoining Land Purchase
        Family: Sale between family members
        Partial: Home was not completed when last assessed
    BUILDTYPE – The type of building
        House
        Commercial
        Others

Surrounding and Locality

    AREA – The property in which the real estate is located
    DIST_MAINROAD – The distance of the property to the main road
    PARK_FACIL – Whether parking facility is available
    UTILITY_AVAIL
        AllPub: All public Utilities (E,G,W,& S)
        NoSewr: Electricity, Gas, and Water (Septic Tank)
        NoSeWa: Electricity and Gas Only
        ELO: Electricity only
    STREET
        Gravel
        Paved
        No Access
    MZZONE
        A: Agriculture
        C: Commercial
        I: Industrial
        RH: Residential High Density
        RL: Residential Low Density
        RM: Residential Medium Density

House Sale Price

    PRT_ID – The Property Transaction ID assigned by ChennaiEstate
    COMMIS – The Commission paid to the agent
    SALES_PRICE – The total sale price of the property



In [2]:
import warnings
import pandas as pd
import numpy as np
from joblib import load
from babel.numbers import format_decimal
from decimal import Decimal
import ipywidgets as widgets
from IPython.display import display

# Load the model
model = load("/content/drive/MyDrive/Estate/House_price_prediction.joblib")

# Define the categorical attribute names
categorical_attributes = ['N_BEDROOMS', 'N_BATHROOM', 'N_ROOM', 'AREA', 'SALE_COND', 'PARK_FACIL', 'BUILDTYPE', 'UTILITY_AVAIL', 'STREET', 'MZZONE']

# Define the unique values for each categorical attribute
unique_values = {
    'N_BEDROOMS': [1, 2, 3, 4],
    'N_BATHROOM': [1, 2],
    'N_ROOM': [2, 3, 4, 5, 6],
    'AREA': ['AREA_Adyar', 'AREA_Anna_Nagar', 'AREA_Chrompet', 'AREA_KK_Nagar', 'AREA_Karapakkam', 'AREA_T_Nagar', 'AREA_Velachery'],
    'SALE_COND': ['SALE_COND_AbNormal', 'SALE_COND_AdjLand', 'SALE_COND_Family', 'SALE_COND_Normal_Sale', 'SALE_COND_Partial'],
    'PARK_FACIL': ['PARK_FACIL_No', 'PARK_FACIL_Yes'],
    'BUILDTYPE': ['BUILDTYPE_Commercial', 'BUILDTYPE_House', 'BUILDTYPE_Others'],
    'UTILITY_AVAIL': ['UTILITY_AVAIL_AllPub', 'UTILITY_AVAIL_ELO', 'UTILITY_AVAIL_NoSeWa', 'UTILITY_AVAIL_NoSewr'],
    'STREET': ['STREET_Gravel', 'STREET_No_Access', 'STREET_Paved'],
    'MZZONE': ['MZZONE_A', 'MZZONE_C', 'MZZONE_I', 'MZZONE_RH', 'MZZONE_RL', 'MZZONE_RM']
}

# Create input widgets for each attribute
input_widgets = {}

for attribute in categorical_attributes:
    input_widgets[attribute] = widgets.Dropdown(options=unique_values[attribute], description=attribute)

input_widgets['INT_SQFT'] = widgets.IntText(description='INT_SQFT:')
input_widgets['DIST_MAINROAD'] = widgets.IntText(description='DIST_MAINROAD:')
input_widgets['QS_ROOMS'] = widgets.FloatText(description='QS_ROOMS:')
input_widgets['QS_BATHROOM'] = widgets.FloatText(description='QS_BATHROOM:')
input_widgets['QS_BEDROOM'] = widgets.FloatText(description='QS_BEDROOM:')
input_widgets['QS_OVERALL'] = widgets.FloatText(description='QS_OVERALL:')
input_widgets['COMMIS'] = widgets.FloatText(description='COMMIS:')

# Create the "Predict" button
predict_button = widgets.Button(description="Predict")

# Create an output label
output_label = widgets.Label()

# Define a function to handle the prediction
def predict_button_click(change):
    warnings.filterwarnings("ignore")
    input_data = {attribute: widget.value for attribute, widget in input_widgets.items()}
    data = pd.DataFrame([input_data])
    binary_encoded_data = pd.DataFrame()

    for attribute in categorical_attributes:
        for value in unique_values[attribute]:
            binary_encoded_data[f'{attribute}_{value}'] = np.where(data[attribute] == value, 1, 0)

    generated_attributes = ['INT_SQFT', 'DIST_MAINROAD', 'QS_ROOMS', 'QS_BATHROOM', 'QS_BEDROOM',
                            'QS_OVERALL', 'COMMIS'] + binary_encoded_data.columns.tolist()

    final_data = pd.concat([data.drop(columns=categorical_attributes), binary_encoded_data], axis=1)[generated_attributes]
    s = [np.array(final_data).reshape(-1)]
    q = model.predict(s)

    v = int(q)
    value = Decimal(v)
    formatted_value = format_decimal(value, locale='en_IN')
    output_label.value = f'Predicted Price: {formatted_value}'

# Attach the function to the button click event
predict_button.on_click(predict_button_click)

# Display widgets
input_box = widgets.VBox(list(input_widgets.values()))
display(input_box, predict_button, output_label)


Button(description='Predict', style=ButtonStyle())

Label(value='')